# Title

### Group 18

Members: Antonie Hille, Bram Dorland, Michael Liem, Nynke Sterk, Gergely Turi​

GitHub repository: https://github.com/mtliem98/TIL6022_projectgroup_18

# Contribution Statement

Antonie Hille:

Bram Dorland:

Michael Liem:

Nynke Sterk:

Gergely Turi​:

# Introduction

Modelling the effect of building the Lelylijn on line usage in the surrounding area.

# Research Questions

How many people travel between each station currently?

See how the newly built line would affect the travel time map.

How many people have better access to big transport hubs?

How many passengers will use the Lelylijn without any additional demand?​

How does the passenger flow change with the Lelylijn on the surrounding train lines without any additional demand?​

# Data Used

For our exercise we used two main sources of data, along with a other minor ones. To build our OD matrix, we used the publicly available NS Traveller behaviour dataset. This includes the yearly workday averages of different passenger metrics on all NS operated railway lines. At time of submission, it is accessible on the following link: https://dashboards.nsjaarverslag.nl/reizigersgedrag

![alt text](Images/dashboard.png)

To get the locations of the statios and the median travel time between them, our program automatically fetches the latest GTFS database from https://gtfs.ovapi.nl/nl/gtfs-nl.zip and compares it with the local copy (if it already exists).

As for Lelylinj's proposed travel times, we used the following map, found in the following .pdf file, on Lelylijn's website: https://delelylijn.nl/wp-content/uploads/2024/05/28-Meibrief-Lelylijn_280524.pdf

![alt text](<Images/new travel times.png>)

# Data Pipeline

Import data on all stations and lines.

Import GTFS dataset.

Create L and P-graph.

Use a shortest path algorithm to determine the cost of travelling between two places.

Model travel behaviour and create an OD-matrix.

Verify the above model with the directional data from the NS.

Calculate the projected change in travel times (and create maps and charts on it).

Apply the same travel behaviour model to the new map with the same number of passengers.

# Scale

To keep our workload within reasonable limits, we had to limit the scope of our work, both in terms of geographical and temporal means

### Geographical

To keep the input data relevant to Lelylijn, we only included station data within the following geographical limits: 

* East of Amsterdam​

* Northeast of Utrecht​

* North of Deventer​

This way we made sure the theoretical construction the new high-speed railway line wil not affect the passenger flow situated far from the new piece of infrastructure.

![alt text](<Images/geo limits.png>)

### Temporal

As for the number of passengers, we used the most recent data available on the NS website (year 2024). We applied the numbers gathered from this source all the way through the entire exercise.

In reality the construction will draw more travelers to trains, changing the modal split and raising the line usage. We did not account for this effect, we applied the already mentioned demand data to the new line.

# Data visualization

table with big stations, travel time change (heatmap/table) (Antonie, Gergő)
big stations: Groningen, Leeuwarden, Zwolle, Lelystad, Amsterdam grouped, Utrecht Centraal, Amersfoort Centraal, Weesp

map of edge saturation near Zwolle

change of demand per edge (as the demand is mostly centered around AMS)

# Answers to the questions here?

# Conclusion

As we have previously demonstrated, the construction of Lelylijn would 